In [5]:
# Install required packages
!pip install requests beautifulsoup4 pandas openpyxl

# Import necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os
from google.colab import files
import getpass

class ScreenerDownloader:
    def __init__(self, username=None, password=None):
        self.base_url = "https://www.screener.in"
        self.session = requests.Session()
        self.logged_in = False

        # If credentials provided, login
        if username and password:
            self.login(username, password)

    def login(self, username=None, password=None):
        """Log in to Screener.in"""
        if not username:
            username = input("Enter your Screener.in username/email: ")
        if not password:
            password = getpass.getpass("Enter your Screener.in password: ")

        login_url = f"{self.base_url}/login/"

        # First get the CSRF token
        response = self.session.get(login_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        csrf_token = soup.find('input', {'name': 'csrfmiddlewaretoken'}).get('value')

        # Now login
        login_data = {
            'csrfmiddlewaretoken': csrf_token,
            'username': username,
            'password': password,
            'next': '/'
        }

        headers = {
            'Referer': login_url,
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        response = self.session.post(login_url, data=login_data, headers=headers)

        # Check if login was successful
        if "Login - " not in response.text:
            print("Login successful!")
            self.logged_in = True
        else:
            print("Login failed. Please check your credentials.")
            self.logged_in = False

    def search_company(self, company_name):
        """Search for a company and return its URL"""
        search_url = f"{self.base_url}/company/{company_name}/"

        # Try direct URL first (if company_name is actually a URL-friendly company name)
        response = self.session.get(search_url)
        if response.status_code == 200 and "is not found" not in response.text:
            return search_url

        # If direct URL doesn't work, try search
        search_url = f"{self.base_url}/api/company/search/?q={company_name}"
        response = self.session.get(search_url)

        if response.status_code == 200:
            results = response.json()
            if results and len(results) > 0:
                # Get the first result
                company_url = results[0].get('url')
                if company_url:
                    return f"{self.base_url}{company_url}"

        print(f"Company '{company_name}' not found.")
        return None

    def get_company_data(self, company_url):
        """Extract financial data from company page"""
        print(f"Fetching data from {company_url}")
        response = self.session.get(company_url)

        if response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code}")
            return None

        soup = BeautifulSoup(response.text, 'html.parser')

        # Get company name
        company_name_elem = soup.find('h1')
        if company_name_elem:
            company_name = company_name_elem.text.strip()
        else:
            # Fallback if h1 is not found
            title = soup.find('title')
            company_name = title.text.strip() if title else "Unknown Company"

        # Get all tables on the page
        data = {}
        tables = soup.find_all('table')

        for table in tables:
            # Try to find the table title in a nearby h2 or h3
            table_title = None
            element = table.find_previous(['h2', 'h3', 'h4'])
            if element:
                table_title = element.text.strip()
            else:
                # If no specific title found, use "Table X"
                table_title = f"Table {len(data) + 1}"

            # Get table data
            table_data = self._extract_table_data(table)
            if table_data is not None:
                data[table_title] = table_data

        # Get ratios and metrics from the page
        ratios = {}

        # Try different selectors as the website might have changed
        ratio_elements = soup.select('div.flex-row span.flex-1') or soup.select('.financial-ratios .data-card')

        for element in ratio_elements:
            try:
                # Try different class names for label and value
                label_elem = element.find('span', class_='name') or element.find('span', class_='flex-1') or element.find('div', class_='name')
                value_elem = element.find('span', class_='number') or element.find('span', class_='font-medium') or element.find('div', class_='value')

                if label_elem and value_elem:
                    label = label_elem.text.strip()
                    value = value_elem.text.strip()
                    ratios[label] = value
            except Exception as e:
                print(f"Error extracting ratio: {e}")
                continue

        data['Key Ratios'] = ratios

        # Get company basic info - try multiple selector patterns as site may have changed
        company_info = {}

        # Try different selectors for company info sections
        info_sections = soup.select('div.company-ratios > div.flex.flex-wrap') or soup.select('.company-info .flex.flex-wrap')

        if info_sections:
            for section in info_sections:
                # Try different class patterns for labels and values
                labels = section.select('span.font-normal') or section.select('.label')
                values = section.select('span.font-semibold') or section.select('.value')

                for i in range(min(len(labels), len(values))):
                    label = labels[i].text.strip()
                    value = values[i].text.strip()
                    company_info[label] = value
        else:
            # Alternative method - look for key-value pairs
            info_rows = soup.select('.company-ratios li') or soup.select('.company-info li')
            for row in info_rows:
                try:
                    text = row.get_text(strip=True)
                    if ':' in text:
                        label, value = text.split(':', 1)
                        company_info[label.strip()] = value.strip()
                except:
                    continue

        data['Company Info'] = company_info

        return {'company_name': company_name, 'data': data}

    def _extract_table_data(self, table):
        """Extract data from a table into a pandas DataFrame"""
        rows = table.find_all('tr')
        if not rows:
            return None

        # Get all headers
        headers = []
        header_row = rows[0]
        for th in header_row.find_all(['th', 'td']):
            headers.append(th.text.strip())

        # Get all data rows
        data = []
        for row in rows[1:]:
            cols = row.find_all(['th', 'td'])
            row_data = []
            for col in cols:
                row_data.append(col.text.strip())
            data.append(row_data)

        # Handle empty data
        if not data:
            return pd.DataFrame(columns=headers)

        # Ensure all rows have the same length as headers
        max_cols = max(len(row) for row in data)
        header_count = len(headers)

        # Adjust headers if needed
        if max_cols > header_count:
            headers.extend([f'Column {i+1}' for i in range(header_count, max_cols)])

        # Create DataFrame with the right number of columns
        df = pd.DataFrame(data)

        # Ensure we have the right number of column names
        column_names = headers[:len(df.columns)]
        if len(column_names) < len(df.columns):
            column_names.extend([f'Column {i+1}' for i in range(len(column_names), len(df.columns))])

        df.columns = column_names
        return df

    def save_data(self, data, output_file=None):
        """Save company data to Excel file"""
        if not data:
            print("No data to save.")
            return

        company_name = data['company_name']

        # Clean company name for use in filename
        clean_name = re.sub(r'[^\w\s-]', '', company_name).strip().replace(' ', '_')

        if not output_file:
            output_file = f"{clean_name}_data.xlsx"

        with pd.ExcelWriter(output_file) as writer:
            # Create a summary sheet
            summary = pd.DataFrame(list(data['data'].get('Key Ratios', {}).items()),
                                   columns=['Ratio', 'Value'])
            summary.to_excel(writer, sheet_name='Summary', index=False)

            # Create a company info sheet
            company_info = pd.DataFrame(list(data['data'].get('Company Info', {}).items()),
                                      columns=['Info', 'Value'])
            company_info.to_excel(writer, sheet_name='Company Info', index=False)

            # Add other data tables
            for table_name, table_data in data['data'].items():
                if table_name in ['Key Ratios', 'Company Info']:
                    continue

                if isinstance(table_data, pd.DataFrame):
                    # Truncate sheet name if too long for Excel
                    sheet_name = table_name[:31] if len(table_name) > 31 else table_name
                    table_data.to_excel(writer, sheet_name=sheet_name, index=False)

        print(f"Data saved to {output_file}")
        # For Colab: Download the file automatically
        try:
            files.download(output_file)
            print(f"File download initiated for {output_file}")
        except:
            print(f"Please download the file manually from the Colab file browser")
        return output_file

    def download_company_data(self, company_name, output_file=None):
        """Complete workflow to download and save company data"""
        company_url = self.search_company(company_name)
        if not company_url:
            return None

        company_data = self.get_company_data(company_url)
        if not company_data:
            return None

        output_file = self.save_data(company_data, output_file)
        return output_file

# Define a more robust function to handle errors gracefully
def safe_download_company(downloader, company_name):
    try:
        print(f"\nProcessing {company_name}...")
        return downloader.download_company_data(company_name)
    except Exception as e:
        print(f"Error processing {company_name}: {str(e)}")
        return None

def download_from_excel(downloader, excel_file):
    """Download data for companies listed in an Excel file"""
    print(f"Reading company names from: {excel_file}")
    try:
        # Read the Excel file
        df = pd.read_excel(excel_file)

        # Look for a column with company names
        company_col = None
        possible_column_names = ['company', 'company name', 'name', 'companies', 'stock']

        # Find the first column that matches our expected naming patterns
        for col in df.columns:
            if col.lower() in possible_column_names or any(name in col.lower() for name in possible_column_names):
                company_col = col
                break

        # If we can't find a likely column, use the first column
        if company_col is None:
            company_col = df.columns[0]
            print(f"Using first column '{company_col}' for company names")
        else:
            print(f"Using column '{company_col}' for company names")

        # Extract the company names
        companies = df[company_col].dropna().tolist()
        print(f"Found {len(companies)} companies to process")

        # Create a new directory for all downloads
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        output_dir = f"screener_data_{timestamp}"
        os.makedirs(output_dir, exist_ok=True)
        print(f"Created output directory: {output_dir}")

        # Download data for each company
        successful = 0
        failed = 0
        for i, company in enumerate(companies):
            print(f"\nProcessing company {i+1}/{len(companies)}: {company}")
            output_file = os.path.join(output_dir, f"{company.replace(' ', '_')}_data.xlsx")

            try:
                result = downloader.download_company_data(company, output_file)
                if result:
                    successful += 1
                else:
                    failed += 1
            except Exception as e:
                print(f"Error: {str(e)}")
                failed += 1

            # Add a delay to avoid overloading the server
            time.sleep(2)

        print(f"\nProcessing complete. Successful: {successful}, Failed: {failed}")

        # Create a zip file of all results
        zip_filename = f"{output_dir}.zip"
        print(f"Creating zip file: {zip_filename}")

        # Use system commands to create a zip file
        !zip -r {zip_filename} {output_dir}

        # Download the zip file
        try:
            files.download(zip_filename)
            print(f"Download initiated for {zip_filename}")
        except Exception as e:
            print(f"Could not auto-download zip file: {str(e)}")
            print(f"Please download {zip_filename} manually from the Colab file browser")

        return True

    except Exception as e:
        print(f"Error processing Excel file: {str(e)}")
        return False

# Example usage - Run this section to download data
try:
    # Create the downloader
    downloader = ScreenerDownloader()

    # Optional: Log in for premium features
    # Uncomment the line below if you want to log in
    # downloader.login()

    # Choose download mode
    print("Choose an option:")
    print("1. Download data for a single company")
    print("2. Download data for multiple companies (enter names)")
    print("3. Upload Excel file with company names")

    choice = input("Enter your choice (1, 2, or 3): ")

    if choice == "1":
        # Download single company
        company_name = input("Enter company name to download data for: ")
        safe_download_company(downloader, company_name)
    elif choice == "2":
        # Download multiple companies
        companies = input("Enter company names separated by commas: ").split(',')
        for company in companies:
            company = company.strip()
            if company:
                safe_download_company(downloader, company)
                # Add a small delay between requests
                time.sleep(2)
    elif choice == "3":
        print("Please upload an Excel file with company names:")
        from google.colab import files
        uploaded = files.upload()

        if uploaded:
            # Get the filename of the uploaded file
            excel_file = list(uploaded.keys())[0]
            download_from_excel(downloader, excel_file)
        else:
            print("No file was uploaded.")
    else:
        print("Invalid choice. Please run again and select 1, 2, or 3.")
except Exception as e:
    print(f"An error occurred: {str(e)}")
    print("Please try again or check if Screener.in website structure has changed.")

Choose an option:
1. Download data for a single company
2. Download data for multiple companies (enter names)
3. Upload Excel file with company names
Enter your choice (1, 2, or 3): 1
Enter company name to download data for: Tata Motors Ltd

Processing Tata Motors Ltd...
Fetching data from https://www.screener.in/company/TATAMOTORS/consolidated/
Data saved to Tata_Motors_Ltd_data.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File download initiated for Tata_Motors_Ltd_data.xlsx
